In [4]:
# Import dependencies
import random
import copy
import numpy as np
import time
import pprint
from itertools import product

In [2]:
# Start the full timer
boot_start = time.time()

# Custom functions

In [3]:
# Takes an input, and randomly selects an index from the original, to one by one assemble a shuffled deck
def randomized_shuffling(master_deck, discard_deck):
    # Port and merge
    temp_deck = []
    temp_deck.extend(discard_deck)
    temp_deck.extend(master_deck)
    
    # Resets
    master_deck = []
    discard_deck = []

    # Randomize the deck
    while len(temp_deck) > 0:
        # Pick a random index
        random_index = random.randint(0, len(temp_deck) - 1)

        master_deck.append(temp_deck.pop(random_index))

    return master_deck, discard_deck

# Takes an input, and shuffles is in an alternating pattern
def interlaced_shuffling(master_deck, discard_deck):
    # Port and merge
    temp_deck = []
    temp_deck.extend(discard_deck)
    temp_deck.extend(master_deck)
    
    # Resets
    master_deck = []
    discard_deck = []

    # Randomize the deck a random number of times
    shuffle_count = random.randint(10, 20)
    for _ in range(shuffle_count):
        # Interlace shuffle
        half = len(temp_deck) // 2
        shuffled_deck = []
        for card in range(half):
            shuffled_deck.append(temp_deck[half + card])
            shuffled_deck.append(temp_deck[card])
        temp_deck = shuffled_deck  # Update temp_deck for the next iteration
    
    # final port
    master_deck = temp_deck

    return master_deck, discard_deck

# Takes 2 inputs, deconstructs them into 52 bins, then reassembles shuffled deck
def rotary_shuffling(master_deck, discard_deck):
    # Port and merge
    temp_deck = []
    temp_deck.extend(discard_deck)
    temp_deck.extend(master_deck)

    # Resets
    master_deck = []
    discard_deck = []

    # Rotary shuffler reset / setup
    rotary_shuffler = {}
    slot_bins = list(range(52))
    open_bins = list(range(52))

    for id in range(52):
        # Building out dictionary with 52 bins
        slot_name = f"Bin {id}"
        rotary_shuffler[slot_name] = []

    # Loop until all cards are binned
    while temp_deck:
        # Randomized target
        current_slot = random.choice(open_bins)
        slot_name = f"Bin {current_slot}"

        # Checking if bin has room, and transferring values if able
        if len(rotary_shuffler[slot_name]) <= 8:
            rotary_shuffler[slot_name].append(temp_deck.pop(0))
            if len(rotary_shuffler[slot_name]) == 8:
                open_bins.remove(current_slot)

    # Dump bins to new master_deck
    opposite_slot = random.choice(slot_bins) + 25

    while slot_bins:
        # Adjustments to stay within correct index range
        opposite_slot %= 52
        slot_name = f"Bin {opposite_slot}"

        # Transferring values back to master_deck
        master_deck.extend(rotary_shuffler[slot_name])

        del rotary_shuffler[slot_name]
        slot_bins.remove(opposite_slot)

        # Move to next bin
        opposite_slot += 1

    # Return the shuffled master_deck and the emptied/reset discard_deck
    return master_deck, discard_deck

# Takes 2 inputs, and shuffles them by serperaing and re-assembling a randomg amount of times
def dual_shuffling(master_deck, discard_deck):
    # Port and merge
    temp_deck = []
    temp_deck.extend(discard_deck)
    temp_deck.extend(master_deck)

    left_temp = []
    right_temp = []

    # Resets
    master_deck = []
    discard_deck = []

    # Random shuffle rounds
    side_shuffles = random.randint(5, 10)

    for side_shuffle in range(side_shuffles):
        # Shuffle loop
        while temp_deck:
            bin_side = random.randint(0, 1)
            if bin_side == 1:
                left_temp.append(temp_deck.pop(0))
            else:
                right_temp.append(temp_deck.pop(0))

        # Merge loop
        while left_temp or right_temp:
            # Merge back left side
            if left_temp:
                master_deck.append(left_temp.pop(0))
            # Merge back right side
            if right_temp:
                master_deck.append(right_temp.pop(0))

    # Return the shuffled master_deck and the emptied/reset discard_deck
    return master_deck, discard_deck

# creates a random number with a random bias within 50 to 100 from the beginning/end
def cut_gen():
    start_bias = random.randint(50, 100)
    end_bias = random.randint(300, 350)
    cut_target = random.randint(start_bias, end_bias)
    return cut_target

# Initial Deck Build

In [4]:
#Inital Deck build
# Unicode icons
spade = '\u2660'
club = '\u2663'
heart = '\u2665'
diamond = '\u2666'

#setting up unique features
suits = [spade,diamond,club,heart]
faces = ["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
values = [11,2,3,4,5,6,7,8,9,10,10,10,10]

# initial setup
pre_deck = {"suits": [],
            "values": []}

# loop to initalize all unique combinations in both bins
for suit_index,suit in enumerate(suits):
    # forward iteration for first 2 suits
    if suit_index < 2:
        for face_index, face in enumerate(faces):
            pre_deck["suits"] += [suit + face]
            pre_deck["values"] += [values[face_index]]
    # reverse iteration for last 2 suits
    else:
        for face_index, face in enumerate(reversed(faces)):
            pre_deck["suits"] += [suit + face]
            pre_deck["values"] += [values[-face_index-1]]
            
# Now building a deck of 8 decks in new deck order
pre_master = {"suits": [],
              "values": [],
              "ids": []}

# loop to make master deck using 8 raw decks
for deck in range(8):
    pre_master["suits"] += pre_deck["suits"]
    pre_master["values"] += pre_deck["values"]
    
# loop to create unique ids for each of the cards made in the master deck
for id in range(len(pre_master["suits"])):
    pre_master["ids"] += [id+1]
    
# Coversion to Tuples
raw_master = []

for card in range(len(pre_master["suits"])):
    raw_master.append((pre_master["suits"][card],
                       pre_master["values"][card],
                       pre_master["ids"][card]))

In [5]:
# initializing discard deck
discard_deck = []
# doing a random shuffle first
master_deck, discard_deck = randomized_shuffling(raw_master, discard_deck)
# then doing an interlaced shuffle to further randomize
master_deck, discard_deck  = interlaced_shuffling(master_deck, discard_deck)

In [6]:
# test prints
print(len(master_deck))
print(master_deck[0:9])

416
[('♥J', 10, 42), ('♦2', 2, 171), ('♠Q', 10, 324), ('♠3', 3, 315), ('♦10', 10, 75), ('♠2', 2, 2), ('♣8', 8, 136), ('♥10', 10, 95), ('♣9', 9, 239)]


In [7]:
#Re-Testing random pairing for accuracy
max_range = len(master_deck)-2

for X in range(10):
    
    first = random.randint(0,max_range)
    second = first+1

    # print test
    print(f"Hand: {X+1}")
    print("   First Card:",master_deck[first][0])
    print("   Second Card:",master_deck[second][0])
    print("   Hand Value:", (master_deck[first][1] + master_deck[second][1]))

Hand: 1
   First Card: ♣6
   Second Card: ♠8
   Hand Value: 14
Hand: 2
   First Card: ♦7
   Second Card: ♥8
   Hand Value: 15
Hand: 3
   First Card: ♦Q
   Second Card: ♦2
   Hand Value: 12
Hand: 4
   First Card: ♠A
   Second Card: ♠7
   Hand Value: 18
Hand: 5
   First Card: ♥3
   Second Card: ♦8
   Hand Value: 11
Hand: 6
   First Card: ♣8
   Second Card: ♣2
   Hand Value: 10
Hand: 7
   First Card: ♣A
   Second Card: ♣2
   Hand Value: 13
Hand: 8
   First Card: ♣3
   Second Card: ♦9
   Hand Value: 12
Hand: 9
   First Card: ♦K
   Second Card: ♦J
   Hand Value: 20
Hand: 10
   First Card: ♣A
   Second Card: ♥8
   Hand Value: 19


# Play Simulator

In [8]:
win_rates = {"PJDL":0, #Player blackjack, dealer lost
             "PJDB":0, #Player blackjack, dealer bust
             "PLDJ":0, #Player lost, dealer blackjack
             "PBDJ":0, #Player bust, dealer blackjack
             "PJDJ":0, #Push, each with blackjacks
             "PWDL":0, #Player wins, dealer lost
             "PWDB":0, #Player wins, dealer bust
             "PLDW":0, #Player loss, dealer wins
             "PBDW":0, #Player bust, dealer wins
             "PBDB":0, #Player bust, dealer bust
             "PpDp":0, #Standard Push
             "Errors":0} #Error Check

# Notes

### XX Convert to while loop for 'while shuffles < X:'
### Make duplicates to test other shuffling methods
### Add baseline betting system with 1.5 payouts for BJs
### Add player card counter that influences draw tactics
### Add true card counter that tracks all cards played
### XX Add initial discard before playing, which isn't considered in player card count
### Try adding 'double until win' betting tactic to assess effectiveness
### Add pandas to make basic graphs to showcase findings
### Add Ace Value flip

In [9]:
# Randomly picks a shuffle trigger between the first and last 50 cards
cut_target = cut_gen()

# Initial Setup
shuffles = 0
games = 0
dealer = []
player = []
last_deck = master_deck.copy()
min_displacement = []
max_displacement = []
average_displacement = []

# Error finding
#error_a = 0
#error_b = 0
#error_c = 0
#error_d = 0
#error_e = 0

# Start the deal timer
deal_start = time.time()

# Loop for X rounds
for rounds in range(1000000):
#while shuffles < 100000:
    # Shuffle if cut target is reached
    if len(master_deck) <= cut_target:
        master_deck, discard_deck = rotary_shuffling(master_deck, discard_deck)
        #master_deck, discard_deck = randomized_shuffling(raw_master, discard_deck)
        #master_deck, discard_deck  = interlaced_shuffling(master_deck, discard_deck)
        
        # Track shuffle effectiveness
        # Create a dictionary mapping values to their index in list1
        value_to_index = {tuple[2]: index for index, tuple in enumerate(last_deck)}

        displacements = []
        for tuple2 in master_deck:
            value2 = tuple2[2]  # Third element of tuple from list2
            index1 = value_to_index.get(value2)  # Get index from dictionary
            if index1 is not None:
                index2 = master_deck.index(tuple2)  # Index in list2
                displacement = index2 - index1
                displacements.append(abs(displacement))

        #print("Displacements:", displacements)

        min_displacement += [min(displacements)]
        max_displacement += [max(displacements)]
        average_displacement += [sum(displacements) / len(displacements)]
        
        # reset
        last_deck = master_deck.copy()
        
        shuffles += 1
        # Randomly picks a shuffle trigger between the first and last 50 cards
        cut_target = cut_gen()
        
    # Initial Draws
    discard_deck.append(tuple(master_deck.pop(0)))
    for deal in range(2):
        dealer.append(tuple(master_deck.pop(0)))
        player.append(tuple(master_deck.pop(0)))
            
    # Check for instant loss/push condition, Skips to results if met
    if sum(card[1] for card in dealer) != 21:
        # Player Play
        while sum(card[1] for card in player) <= 16:
            player.append(tuple(master_deck.pop(0)))

        # Dealer Play
        while sum(card[1] for card in dealer) <= 17:
            dealer.append(tuple(master_deck.pop(0)))
    
    # Round Results
    if sum(card[1] for card in player) == 21 and len(player) == 2:
        # Player blackjack
        if sum(card[1] for card in dealer) == 21:
            if len(dealer) == 2:
                win_rates["PJDJ"] += 1  # Push, each with blackjacks
            else:
                win_rates["PpDp"] += 1  # Standard Push
        elif sum(card[1] for card in dealer) > 21:
            win_rates["PJDB"] += 1  # Player blackjack, dealer bust
        elif sum(card[1] for card in dealer) < 21:
            win_rates["PJDL"] += 1  # Player blackjack, dealer lost
        else:
            win_rates["Errors"] += 1  # Error bin
            #error_a += 1

    elif sum(card[1] for card in dealer) == 21 and len(dealer) == 2:
        # Dealer blackjack
        if sum(card[1] for card in player) != 21:
            win_rates["PLDJ"] += 1  # Player lost, dealer blackjack
        else:
            win_rates["Errors"] += 1  # Error bin
            #error_b += 1

    elif sum(card[1] for card in player) <= 21: 
        # Player valid        
        if sum(card[1] for card in dealer) <= 21:
            if sum(card[1] for card in player) > sum(card[1] for card in dealer):
                win_rates["PWDL"] += 1  # Player wins, dealer lost
            elif sum(card[1] for card in player) < sum(card[1] for card in dealer):
                win_rates["PLDW"] += 1  # Player loss, dealer wins
            elif sum(card[1] for card in dealer) > 21:
                win_rates["PWDB"] += 1  # Player wins, dealer bust
            elif sum(card[1] for card in player) == sum(card[1] for card in dealer):
                win_rates["PpDp"] += 1  # Standard Push
            else:
                win_rates["Errors"] += 1  # Error bin
                #error_c += 1
                
        else:
            win_rates["PWDB"] += 1  # Player wins, dealer bust
            
    else:
        # Player busts
        if sum(card[1] for card in dealer) <= 21:  # Dealer valid
            win_rates["PBDW"] += 1  # Player bust, dealer wins
        elif sum(card[1] for card in dealer) > 21:
            win_rates["PBDB"] += 1  # Player bust, dealer bust
        else:
            win_rates["Errors"] += 1  # Error bin
            #error_d += 1

    # Discard Transfer
    for card in dealer:
        discard_deck.append(card)

    for card in player:
        discard_deck.append(card)
    
    # Bin resets
    dealer = []
    player = []

    games += 1
    
# End the deal timer
deal_end = time.time()

In [10]:
print(len(discard_deck)+len(master_deck))

416


# RESULTS

In [11]:
print(f"Number of games = {games}")
print(f"Number of shuffles = {shuffles}")

Number of games = 3255671
Number of shuffles = 100000


In [12]:
# Calculating sum of results
total_plays = 0
for key in win_rates:
    total_plays += win_rates[key]

print(f"Player blackjack, dealer lost = {win_rates['PJDL']}")
print(f"Player blackjack, dealer bust = {win_rates['PJDB']}")
print(f"Player lost, dealer blackjack = {win_rates['PLDJ']}")
print(f"Player bust, dealer blackjack = {win_rates['PBDJ']}")
print(f"Push, each with blackjacks = {win_rates['PJDJ']}")
print(f"Player wins, dealer lost = {win_rates['PWDL']}")
print(f"Player wins, dealer bust = {win_rates['PWDB']}")
print(f"Player loss, dealer wins = {win_rates['PLDW']}")
print(f"Player bust, dealer wins = {win_rates['PBDW']}")
print(f"Player bust, dealer bust = {win_rates['PBDB']}")
print(f"Standard Push = {win_rates['PpDp']}")
print(f"Error Count = {win_rates['Errors']}")
print("") # space
print(f"Total results = {total_plays}")

Player blackjack, dealer lost = 69677
Player blackjack, dealer bust = 65639
Player lost, dealer blackjack = 147156
Player bust, dealer blackjack = 0
Push, each with blackjacks = 7054
Player wins, dealer lost = 304158
Player wins, dealer bust = 869153
Player loss, dealer wins = 540921
Player bust, dealer wins = 559603
Player bust, dealer bust = 448820
Standard Push = 243490
Error Count = 0

Total results = 3255671


In [13]:
win_rate = (win_rates['PJDL']+
            win_rates['PJDB']+
            win_rates['PWDL']+
            win_rates['PWDB'])
print(f"Win Rate = {(win_rate/total_plays)*100}%")

push_rate = (win_rates['PJDJ']+
            win_rates['PpDp'])
print(f"Push Rate = {(push_rate/total_plays)*100}%")

loss_rate = total_plays-(push_rate+win_rate)
print(f"Loss Rate = {(loss_rate/total_plays)*100}%")

Win Rate = 40.19530843257811%
Push Rate = 7.695617892594184%
Loss Rate = 52.10907367482771%


In [14]:
#print(error_a)  
#print(error_b)
#print(error_c)
#print(error_d)
#print(error_e)

In [15]:
# End the full timer
boot_end = time.time()

# Calculate the elapsed times
deal_time = deal_end - deal_start
boot_time = boot_end - boot_start

# Print the elapsed time
print(f"{games} deals took: {deal_time} seconds")
print(f"Full runtime: {boot_time} seconds")

3255671 deals took: 278.32875752449036 seconds
Full runtime: 278.53099179267883 seconds


In [16]:
#BJ21f 
#1m deals = 17.59 seconds
#Full run = 17.80 seconds
#100k shuffles = 274.00 seconds

In [17]:
dummy = []
list1 = raw_master.copy()
#list2 = randomized_shuffling(raw_master)
#list2 = interlaced_shuffling(raw_master)
list2, _ = rotary_shuffling(raw_master, dummy)

In [18]:
print("Minimum Displacement:", sum(min_displacement) / len(min_displacement))
print("Maximum Displacement:", sum(max_displacement) / len(max_displacement))
print("Average Displacement:", sum(average_displacement) / len(average_displacement))

Minimum Displacement: 0.42995
Maximum Displacement: 392.17202
Average Displacement: 137.62623360576933


In [19]:
count = 0
dummies = [('♥J', 10, 302), 
           ('♦3', 3, 172), 
           ('♣6', 6, 346), 
           ('♣7', 7, 397), 
           ('♦Q', 10, 337), 
           ('♥10', 10, 355), 
           ('♥J', 10, 354), 
           ('♥9', 9, 304), 
           ('♥7', 7, 202)]

In [20]:
test = (dummy[1] for dummy in dummies if dummy[1] >= 10)
test2 = (dummy[1] for dummy in dummies if dummy[1] <= 6)

In [21]:
print(test)

<generator object <genexpr> at 0x00000210E6E2A5E8>


In [22]:
dummies[0][1]

10

In [23]:
for dummy in dummies:
    if dummy[1] >= 10:
        count -= 1
    elif dummy[1] <= 6:
        count += 1

In [24]:
print(count)

-2


In [3]:
def generate_structure(depth, branching_factor, end_branches):
    if depth == 0:
        # Base case: return end branches
        return end_branches
    else:
        # Recursive case: generate branches
        branches = {}
        for i in range(branching_factor):
            branch_name = chr(65 + i)  # Convert index to alphabet character
            branches[branch_name] = generate_structure(depth - 1, branching_factor, end_branches)
        return branches

# Define parameters
depth = 3  # Depth of the structure
branching_factor = 4  # Number of secondary branches per primary branch
end_branches = ["End1", "End2", "End3", "End4"]  # End branches at the desired depth

# Generate the structure
structure = generate_structure(depth, branching_factor, end_branches)

# Print the generated structure
pprint.pprint(structure)

{'A': {'A': {'A': ['End1', 'End2', 'End3', 'End4'],
             'B': ['End1', 'End2', 'End3', 'End4'],
             'C': ['End1', 'End2', 'End3', 'End4'],
             'D': ['End1', 'End2', 'End3', 'End4']},
       'B': {'A': ['End1', 'End2', 'End3', 'End4'],
             'B': ['End1', 'End2', 'End3', 'End4'],
             'C': ['End1', 'End2', 'End3', 'End4'],
             'D': ['End1', 'End2', 'End3', 'End4']},
       'C': {'A': ['End1', 'End2', 'End3', 'End4'],
             'B': ['End1', 'End2', 'End3', 'End4'],
             'C': ['End1', 'End2', 'End3', 'End4'],
             'D': ['End1', 'End2', 'End3', 'End4']},
       'D': {'A': ['End1', 'End2', 'End3', 'End4'],
             'B': ['End1', 'End2', 'End3', 'End4'],
             'C': ['End1', 'End2', 'End3', 'End4'],
             'D': ['End1', 'End2', 'End3', 'End4']}},
 'B': {'A': {'A': ['End1', 'End2', 'End3', 'End4'],
             'B': ['End1', 'End2', 'End3', 'End4'],
             'C': ['End1', 'End2', 'End3', 'End4'],
       

In [5]:
def generate_structure(depth, branching_factors, bin_ranges):
    if depth == 0:
        # Base case: return end branches with unique combinations of values
        return list(product(*[range(start, end + 1) for start, end in bin_ranges]))
    else:
        # Recursive case: generate branches
        branches = {}
        for i in range(branching_factors[depth - 1]):
            branch_name = chr(65 + i)  # Convert index to alphabet character
            branches[branch_name] = generate_structure(depth - 1, branching_factors, bin_ranges)
        return branches

# Define parameters
depth = 3  # Depth of the structure
branching_factors = [2, 3, 4]  # Number of secondary branches per level
bin_ranges = [(2, 21), (2, 11), (1, 4), (1, 4)]  # Range of values for each bin

# Generate the structure
structure = generate_structure(depth, branching_factors, bin_ranges)

# Print the generated structure
pprint.pprint(structure)


{'A': {'A': {'A': [(2, 2, 1, 1),
                   (2, 2, 1, 2),
                   (2, 2, 1, 3),
                   (2, 2, 1, 4),
                   (2, 2, 2, 1),
                   (2, 2, 2, 2),
                   (2, 2, 2, 3),
                   (2, 2, 2, 4),
                   (2, 2, 3, 1),
                   (2, 2, 3, 2),
                   (2, 2, 3, 3),
                   (2, 2, 3, 4),
                   (2, 2, 4, 1),
                   (2, 2, 4, 2),
                   (2, 2, 4, 3),
                   (2, 2, 4, 4),
                   (2, 3, 1, 1),
                   (2, 3, 1, 2),
                   (2, 3, 1, 3),
                   (2, 3, 1, 4),
                   (2, 3, 2, 1),
                   (2, 3, 2, 2),
                   (2, 3, 2, 3),
                   (2, 3, 2, 4),
                   (2, 3, 3, 1),
                   (2, 3, 3, 2),
                   (2, 3, 3, 3),
                   (2, 3, 3, 4),
                   (2, 3, 4, 1),
                   (2, 3, 4, 2),
          

 'B': {'A': {'A': [(2, 2, 1, 1),
                   (2, 2, 1, 2),
                   (2, 2, 1, 3),
                   (2, 2, 1, 4),
                   (2, 2, 2, 1),
                   (2, 2, 2, 2),
                   (2, 2, 2, 3),
                   (2, 2, 2, 4),
                   (2, 2, 3, 1),
                   (2, 2, 3, 2),
                   (2, 2, 3, 3),
                   (2, 2, 3, 4),
                   (2, 2, 4, 1),
                   (2, 2, 4, 2),
                   (2, 2, 4, 3),
                   (2, 2, 4, 4),
                   (2, 3, 1, 1),
                   (2, 3, 1, 2),
                   (2, 3, 1, 3),
                   (2, 3, 1, 4),
                   (2, 3, 2, 1),
                   (2, 3, 2, 2),
                   (2, 3, 2, 3),
                   (2, 3, 2, 4),
                   (2, 3, 3, 1),
                   (2, 3, 3, 2),
                   (2, 3, 3, 3),
                   (2, 3, 3, 4),
                   (2, 3, 4, 1),
                   (2, 3, 4, 2),
          

             'B': [(2, 2, 1, 1),
                   (2, 2, 1, 2),
                   (2, 2, 1, 3),
                   (2, 2, 1, 4),
                   (2, 2, 2, 1),
                   (2, 2, 2, 2),
                   (2, 2, 2, 3),
                   (2, 2, 2, 4),
                   (2, 2, 3, 1),
                   (2, 2, 3, 2),
                   (2, 2, 3, 3),
                   (2, 2, 3, 4),
                   (2, 2, 4, 1),
                   (2, 2, 4, 2),
                   (2, 2, 4, 3),
                   (2, 2, 4, 4),
                   (2, 3, 1, 1),
                   (2, 3, 1, 2),
                   (2, 3, 1, 3),
                   (2, 3, 1, 4),
                   (2, 3, 2, 1),
                   (2, 3, 2, 2),
                   (2, 3, 2, 3),
                   (2, 3, 2, 4),
                   (2, 3, 3, 1),
                   (2, 3, 3, 2),
                   (2, 3, 3, 3),
                   (2, 3, 3, 4),
                   (2, 3, 4, 1),
                   (2, 3, 4, 2),
          

       'C': {'A': [(2, 2, 1, 1),
                   (2, 2, 1, 2),
                   (2, 2, 1, 3),
                   (2, 2, 1, 4),
                   (2, 2, 2, 1),
                   (2, 2, 2, 2),
                   (2, 2, 2, 3),
                   (2, 2, 2, 4),
                   (2, 2, 3, 1),
                   (2, 2, 3, 2),
                   (2, 2, 3, 3),
                   (2, 2, 3, 4),
                   (2, 2, 4, 1),
                   (2, 2, 4, 2),
                   (2, 2, 4, 3),
                   (2, 2, 4, 4),
                   (2, 3, 1, 1),
                   (2, 3, 1, 2),
                   (2, 3, 1, 3),
                   (2, 3, 1, 4),
                   (2, 3, 2, 1),
                   (2, 3, 2, 2),
                   (2, 3, 2, 3),
                   (2, 3, 2, 4),
                   (2, 3, 3, 1),
                   (2, 3, 3, 2),
                   (2, 3, 3, 3),
                   (2, 3, 3, 4),
                   (2, 3, 4, 1),
                   (2, 3, 4, 2),
          

In [10]:
BinA = 2
BinB = 2
BinC = 1

Results = {"2_2_1":{"A": [],
                    "B": [],
                    "C": []},
          ## to
           "21_11_4":{"A": [],
                      "B": [],
                      "C": []}}

While BinA <21:
    While BinB <11:
        While BinC <4:
            ## Test
            ## ResultA
            ## ResultB
            ## ResultC
            ## Save Result into unique slot: [BinA+"_"+BinB+"_"+BinC] = {"A":ResultA, "B":ResultB, "C": ResultC}
            BinC += 1
        BinB += 1
        BinC = 1 ## Reset 1 Bin in
    BinA += 1
    BinB = 2 ## Reset 1 Bin in

SyntaxError: invalid syntax (3348094171.py, line 11)

In [11]:
results = {}

# Define the ranges for each bin
range_binA = range(2, 22)  # Range for BinA: 2 to 21
range_binB = range(2, 12)  # Range for BinB: 2 to 11
range_binC = range(1, 5)    # Range for BinC: 1 to 4

# Iterate over the combinations of values for each bin
for binA in range_binA:
    for binB in range_binB:
        for binC in range_binC:
            # Generate unique key for the combination of bins
            key = f"{binA}_{binB}_{binC}"

            # Perform test, calculate results
            resultA = binA * 2
            resultB = binB * 3
            resultC = binC * 4

            # Save results into the dictionary
            results[key] = {"A": resultA, "B": resultB, "C": resultC}

# Print the results dictionary
print(results)

{'2_2_1': {'A': 4, 'B': 6, 'C': 4}, '2_2_2': {'A': 4, 'B': 6, 'C': 8}, '2_2_3': {'A': 4, 'B': 6, 'C': 12}, '2_2_4': {'A': 4, 'B': 6, 'C': 16}, '2_3_1': {'A': 4, 'B': 9, 'C': 4}, '2_3_2': {'A': 4, 'B': 9, 'C': 8}, '2_3_3': {'A': 4, 'B': 9, 'C': 12}, '2_3_4': {'A': 4, 'B': 9, 'C': 16}, '2_4_1': {'A': 4, 'B': 12, 'C': 4}, '2_4_2': {'A': 4, 'B': 12, 'C': 8}, '2_4_3': {'A': 4, 'B': 12, 'C': 12}, '2_4_4': {'A': 4, 'B': 12, 'C': 16}, '2_5_1': {'A': 4, 'B': 15, 'C': 4}, '2_5_2': {'A': 4, 'B': 15, 'C': 8}, '2_5_3': {'A': 4, 'B': 15, 'C': 12}, '2_5_4': {'A': 4, 'B': 15, 'C': 16}, '2_6_1': {'A': 4, 'B': 18, 'C': 4}, '2_6_2': {'A': 4, 'B': 18, 'C': 8}, '2_6_3': {'A': 4, 'B': 18, 'C': 12}, '2_6_4': {'A': 4, 'B': 18, 'C': 16}, '2_7_1': {'A': 4, 'B': 21, 'C': 4}, '2_7_2': {'A': 4, 'B': 21, 'C': 8}, '2_7_3': {'A': 4, 'B': 21, 'C': 12}, '2_7_4': {'A': 4, 'B': 21, 'C': 16}, '2_8_1': {'A': 4, 'B': 24, 'C': 4}, '2_8_2': {'A': 4, 'B': 24, 'C': 8}, '2_8_3': {'A': 4, 'B': 24, 'C': 12}, '2_8_4': {'A': 4, 'B':

In [12]:
len(results)

800

In [13]:
import pandas as pd

# Initialize dictionary to store highest "A" values
highest_a_values = {}

# Iterate through results and update dictionary
for key, value in results.items():
    bin_a, bin_b, bin_c = key.split('_')
    bin_a = int(bin_a)
    bin_b = int(bin_b)
    bin_c = int(bin_c)
    
    if (bin_a, bin_b) not in highest_a_values or value["A"] > highest_a_values[(bin_a, bin_b)]:
        highest_a_values[(bin_a, bin_b)] = (bin_c, value["A"])

# Create data table
max_a_df = pd.DataFrame(index=range(min(highest_a_values.keys())[0], max(highest_a_values.keys())[0] + 1),
                        columns=range(min(highest_a_values.keys(), key=lambda x: x[1])[1], max(highest_a_values.keys(), key=lambda x: x[1])[1] + 1))

# Populate data table with highest "A" values
for key, value in highest_a_values.items():
    max_a_df.at[key[0], key[1]] = value[0]

print(max_a_df)


TypeError: '>' not supported between instances of 'int' and 'tuple'

# Experimental class version

In [1]:
import random

class Card:
    def __init__(self, suit, value):
        self.suit = suit
        self.value = value

class Deck:
    def __init__(self):
        self.cards = []
        self.build()

    def build(self):
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
        for suit in suits:
            for value in values:
                self.cards.append(Card(suit, value))

    def shuffle(self):
        random.shuffle(self.cards)

    def draw(self):
        if len(self.cards) == 0:
            return None
        return self.cards.pop()

class Player:
    def __init__(self, name):
        self.name = name
        self.hand = []

    def add_card(self, card):
        self.hand.append(card)

    def clear_hand(self):
        self.hand = []

class BlackjackGame:
    def __init__(self, num_rounds):
        self.num_rounds = num_rounds
        self.deck = Deck()
        self.players = []
        self.initialize_players()

    def initialize_players(self):
        num_players = int(input("Enter the number of players: "))
        for i in range(num_players):
            player_name = input(f"Enter name for player {i+1}: ")
            self.players.append(Player(player_name))

    def play_round(self):
        for player in self.players:
            card = self.deck.draw()
            if card:
                player.add_card(card)
            else:
                print("No more cards in the deck!")
                break

    def play_game(self):
        for _ in range(self.num_rounds):
            self.deck.shuffle()
            self.play_round()
            # Check round results and record them
            # Implement your win/loss logic here
            for player in self.players:
                player.clear_hand()

if __name__ == "__main__":
    num_rounds = 10  # Set the number of rounds for the simulation
    game = BlackjackGame(num_rounds)
    game.play_game()


Enter the number of players: 2
Enter name for player 1: dummy
Enter name for player 2: dummier
